# Exploration

A notebook used to explore FRED data

## Setup    

In [1]:
# Load environment variables using dotenv

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Setup autoreload of modules
%load_ext autoreload
%autoreload 2


In [3]:
# Get the FRED API client

from fred_data import api

%autoreload 2

fred_client = api.FredApiClient()

## Data Exploration

In [ ]:
# Fetch all releases
from fred_data.api import api_key_placeholder

response = fred_client.get(f"fred/releases?api_key={api_key_placeholder}")
print(response.json())